# Demo 1: SUMMA

# C. Changing options and settings

One of the things that make SUMMA useful is that you can change model options pretty easily (at least more easily than in many other models), while keep all the rest of the model the same. So in this demo, we will inspect some of the options and change some of the settings.

Before you proceed, remember that you want to save your notebooks in a place other than the `cewa564` directory tree, because files you save there may be overwritten.

In this demo we will use the SUMMA setup for the [Reynolds Creek Watershed](https://criticalzone.org/reynolds/). You can install this setup by using the `install_summa_reynolds_site()` function in the `install_summa_model_configurations.ipynb` notebook in the top level directory. Install it before proceeding.

## Setup

We'll import some of the python modules that you are likely to need right up front. Note that you only need to do this once in a notebook. As long as you are working in the same session, the modules will remain loaded for the duration of that session.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pysumma as ps
import copy
import os.path

In [ ]:
summa_executable = 'summa.exe'
case_path = '/home/jovyan/reynolds'
filemanager_path = os.path.join(case_path, 'settings/snow_fileManager_template.txt')

## Default simulation

We can do a quick simulation to make sure everything is in place:

In [ ]:
S = ps.Simulation(executable='summa.exe', filemanager=filemanager_path)
S.start(run_option='local', run_suffix='default')
S.monitor()

and make a quick plot of the snow water equivalent as a function of time.

In [ ]:
S.output.scalarSWE.plot()

## Change the snow layering scheme

Snow models represent a snow pack as one or more layers of snow and keep track of how much water is stored in each layer (snow water equivalent or SWE) as well as other state variables such as temperature. The model then tracks the time evolution of these state variables in each layer as well as the fluxes of water and energy between the layers.

In SUMMA you can choose between two different methods to combine and sub-divide snow layers. This behavior is controlled by the `snowLayers` variable in the [SUMMA decisions file](https://summa.readthedocs.io/en/latest/input_output/SUMMA_input/#infile_model_decisions) and can take one of two values, either `CLM_2010` or `jrdn1991`. The first refers to how snow layers were handled in the Community Land Model (CLM), while the second refers the snow layering scheme in SNTHERM as describe in  Jordan (1991).

In [ ]:
S.decisions.snowLayers.available_options

Among others things, the `CLM_2010` only has a limited number of snow layers. The layers store a maximum amount of SWE and when SWE in a layer exceeds that value, a new layer is created. Similarly, when SWE in a layer falls below a minimum SWE, the layer is merged with a neighboring layer. Once the maximum number of layers has been created, the deepest layer can grow as deep as it needs to be and acts as snow reservoir with unlimited capacity. The `jrdn1991` scheme on the other hand keeps creating additional layers as the snow pack grows in depth.

Here we will do four different simulations:

* SUMMA with `CLM_2010`
* SUMMA with `jrdn1991`
* SUMMA with `CLM_2010`, but we will configure it to act as a two layer model (the default version uses five snow layers)
* SUMMA with `CLM_2010`, a two layer model and a thick top layer.

### SUMMA with CLM_2010

We'll use the simulation object you are already defined above and then copy the simulation object to create the different simulations.

Use a run_suffix that uniquely identifies this run so that you can keep your runs apart, for example CLM_2010, and run the model. The run itself will take about a minute.

In [ ]:
S_CLM_2010 = copy.copy(S)
S_CLM_2010.decisions.snowLayers.set_value('CLM_2010')
S_CLM_2010.start(run_option = 'local', run_suffix="CLM_2010")
S_CLM_2010.monitor()

You can quickly check how the number of snow layers changes by plotting

In [ ]:
S_CLM_2010.output.nSnow.isel(hru=0).plot()

### SUMMA with jrdn1991

Repeat the exercise with the `jrdn1991` option for `snowLayers`. You can examine the differences in the snow layers themselves, but also in other characteristics of snow pack evolution. In SUMMA, all other options will be the same between these two simulations unless you change things explicitly. Here we just show you how to do the simulations.

In [ ]:
S_JRDN_1991 = copy.copy(S)
S_JRDN_1991.decisions.snowLayers.set_value('jrdn1991')
S_JRDN_1991.start(run_option = 'local', run_suffix="JRDN_1991")
S_JRDN_1991.monitor()

and we can again check how the number of snow layers evolves over the duration of the run

In [ ]:
S_JRDN_1991.output.nSnow.isel(hru=0).plot()

In this case there are many more snow layers that the five in SUMMA run with the `CLM_2010`. In this case, new snow layers are generated as more snow falls and they disappear when the snow melts again.

### SUMMA with CLM_2010 configured as a  two layer model

We'll now change the snowLayers option back to `CLM_2010` and configure the model to act as a two layer snow model rather than a five layer snow model. Two layer models are quite common. The surface layer is typically quite thin in these setups to respond quickly to changing atmospheric conditions, while the second layer acts as a snow reservoir with unlimited capacity.

To configure the snow model as a two layer model, we need to change some model parameters, something we have not done so far, so we'll use some other features in `pysumma`. Here, we'll change some settings in the SUMMA input file that contains parameter information, but `pysumma` will take care of that for you behind the scenes. It "knows" where the various input files are from the file manager file that you provided when you created the simulation object.

In [ ]:
S_CLM_2010_2L = copy.copy(S)
S_CLM_2010_2L.decisions.snowLayers.set_value('CLM_2010')

We are going to change some SUMMA parameter values to the following settings:
```
zminLayer3                |     100.0000 |       0.0500 |       0.0500
zminLayer4                |     100.0000 |       0.1000 |       0.1000
zminLayer5                |     100.0000 |       0.2500 |       0.2500
zmaxLayer2_lower          |    1000.0000 |       0.2000 |       0.2000
zmaxLayer3_lower          |    1000.0000 |       0.5000 |       0.5000
zmaxLayer4_lower          |    1000.0000 |       1.0000 |       1.0000
zmaxLayer2_upper          |    1000.0000 |       0.1500 |       0.1500
zmaxLayer3_upper          |    1000.0000 |       0.3000 |       0.3000
zmaxLayer4_upper          |    1000.0000 |       0.7500 |       0.7500
```
For now don't worry about the last two columns (they are not used). The value in the second column determines when a new snow layer is created or when two layers are merged. By creating a second layer with a very large capacity, layers 3, 4, and 5 will never be created and the model acts like a two layer snow model.

We can evaluate what the current values for the these parameters are:

In [ ]:
print(S_CLM_2010_2L.local_param_info)

or if you know which parameter you are looking for, you can directly use

In [ ]:
S_CLM_2010_2L.local_param_info.zminLayer3.value

We'll now change the values to those mentioned earlier

In [ ]:
S_CLM_2010_2L.local_param_info.zminLayer3.set_value([100.0, 0.0500, 0.0500])

one of the benefits of using the access functions like `set_value()` is that `pysumma` ensures that the data is correctly formatted for the SUMMA input files. Compare for example:

In [ ]:
S_CLM_2010_2L.local_param_info.zminLayer3.value

and

In [ ]:
print(S_CLM_2010_2L.local_param_info.zminLayer3)

We'll now change the other values as well:

In [ ]:
S_CLM_2010_2L.local_param_info.zminLayer3.set_value([100., 0.05, 0.05])
S_CLM_2010_2L.local_param_info.zminLayer4.set_value([100., 0.1, 0.1])
S_CLM_2010_2L.local_param_info.zminLayer5.set_value([100., 0.25, 0.25])
S_CLM_2010_2L.local_param_info.zmaxLayer2_lower.set_value([1000., 0.2, 0.2])
S_CLM_2010_2L.local_param_info.zmaxLayer3_lower.set_value([1000., 0.5, 0.5])
S_CLM_2010_2L.local_param_info.zmaxLayer4_lower.set_value([1000., 1., 1.])
S_CLM_2010_2L.local_param_info.zmaxLayer2_upper.set_value([1000., 0.15, 0.15])
S_CLM_2010_2L.local_param_info.zmaxLayer3_upper.set_value([1000., 0.3, 0.3])
S_CLM_2010_2L.local_param_info.zmaxLayer4_upper.set_value([1000., 0.75, 0.75])

**BE AWARE: as we are making these changes, we are modifying the same SUMMA configuration files. If you want to return to the original SUMMA setup, you will need to manually undo these changes (using the same functions) or you can re-install the model configuration.**


In [ ]:
S_CLM_2010_2L.start(run_option = 'local', run_suffix="CLM_2010_2L")
S_CLM_2010_2L.monitor()

In [ ]:
#print(S_CLM_2010_2L.stdout)

In [ ]:
S_CLM_2010_2L.output.nSnow.isel(hru=0).plot()

In [ ]:
S_CLM_2010_2L.output.scalarSWE.isel(hru=0).plot()

### SUMMA with CLM_2010 configured as a two layer model with a thick top layer

We'll do one more run, but now with SUMMA configured with two layers and a thick top layer. To do this, change the local parameters to
```
zminLayer1                |       0.0500 |       0.0500 |       0.0500
zminLayer2                |       0.0200 |       0.0500 |       0.0500
zminLayer3                |     100.0000 |       0.0500 |       0.0500
zminLayer4                |     100.0000 |       0.1000 |       0.1000
zminLayer5                |     100.0000 |       0.2500 |       0.2500
zmaxLayer1_lower          |       0.2000 |       0.0500 |       0.0500
zmaxLayer2_lower          |    1000.0000 |       0.2000 |       0.2000
zmaxLayer3_lower          |    1000.0000 |       0.5000 |       0.5000
zmaxLayer4_lower          |    1000.0000 |       1.0000 |       1.0000
zmaxLayer1_upper          |       0.1500 |       0.0300 |       0.0300
zmaxLayer2_upper          |    1000.0000 |       0.1500 |       0.1500
zmaxLayer3_upper          |    1000.0000 |       0.3000 |       0.3000
zmaxLayer4_upper          |    1000.0000 |       0.7500 |       0.7500

```

In [ ]:
S_CLM_2010_2L_THICK_TOP = copy.copy(S_CLM_2010_2L)

In [ ]:
S_CLM_2010_2L_THICK_TOP.local_param_info.zminLayer1.set_value([0.05, 0.05, 0.05])
S_CLM_2010_2L_THICK_TOP.local_param_info.zminLayer2.set_value([0.02, 0.05, 0.05])
S_CLM_2010_2L_THICK_TOP.local_param_info.zminLayer3.set_value([100., 0.05, 0.05])
S_CLM_2010_2L_THICK_TOP.local_param_info.zminLayer4.set_value([100., 0.1, 0.1])
S_CLM_2010_2L_THICK_TOP.local_param_info.zminLayer5.set_value([100., 0.25, 0.25])
S_CLM_2010_2L_THICK_TOP.local_param_info.zmaxLayer1_lower.set_value([0.2, 0.05, 0.05])
S_CLM_2010_2L_THICK_TOP.local_param_info.zmaxLayer2_lower.set_value([1000., 0.2, 0.2])
S_CLM_2010_2L_THICK_TOP.local_param_info.zmaxLayer3_lower.set_value([1000., 0.5, 0.5])
S_CLM_2010_2L_THICK_TOP.local_param_info.zmaxLayer4_lower.set_value([1000., 1., 1.])
S_CLM_2010_2L_THICK_TOP.local_param_info.zmaxLayer1_upper.set_value([0.15, 0.03, 0.03])
S_CLM_2010_2L_THICK_TOP.local_param_info.zmaxLayer2_upper.set_value([1000., 0.15, 0.15])
S_CLM_2010_2L_THICK_TOP.local_param_info.zmaxLayer3_upper.set_value([1000., 0.3, 0.3])
S_CLM_2010_2L_THICK_TOP.local_param_info.zmaxLayer4_upper.set_value([1000., 0.75, 0.75])

In [ ]:
S_CLM_2010_2L_THICK_TOP.start(run_option = 'local', run_suffix="CLM_2010_2L_THICK_TOP")
S_CLM_2010_2L_THICK_TOP.monitor()

In [ ]:
S_CLM_2010_2L_THICK_TOP.output.nSnow.isel(hru=0).plot()

After you are done, restore the local parameters to the original values, because we may use this setup again (note that you can also just reinstall the model setup).

To examine how the different options affect the snow depth, we can plot all on the same plot:

In [ ]:
plt.figure(figsize=[16,8])
S_CLM_2010.output.scalarSnowDepth.plot(label='CLM_2010: 5-layer')
S_JRDN_1991.output.scalarSnowDepth.plot(label='jrdn1991')
S_CLM_2010_2L.output.scalarSnowDepth.plot(label='CLM_2010: 2-layer')
S_CLM_2010_2L_THICK_TOP.output.scalarSnowDepth.plot(label='CLM_2010: Thick top')
plt.legend()